# **Modelling and Evaluation Notebook**

## Objectives

Answer business requirement 2 and 3:
- The client is interested to tell whether a given plant is Healthy or Powdery
- Provide recommendations for treating plants infected with powdery mildew

## Inputs

- inputs/datasets/cherry-leaves/train
- inputs/datasets/cherry-leaves/validation
- inputs/datasets/cherry-leaves/test
- image shape embeddings

## Outputs
- Images distribution plot in train, validation, and test set
- Image augmentation
- Class indices to change prediction inference in labels
- Machine learning model creation and training
- Save model
- Learning curve plot for model performance
- Model evaluation on pickle file
- Prediction on the random image file

### Additional Comments

Version 1 will be created and try different ways to fit the model to chose which has more accuracy


---
## **Prepare Data Directories**

#### Import libraries


In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread

#### Change working directory

- Change the working directory from its current folder to its parent folder
- Access the current directory with os.getcwd()

In [2]:
import os
current_dir = os.getcwd()
current_dir


'/workspace/mildew-detector/jupyter_notebooks'

Make the parent of the current directory the new current directory

- os.path.dirname() gets the parent directory
- os.chir() defines the new current directory

In [3]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [4]:
work_dir = os.getcwd()
work_dir

'/workspace/mildew-detector'

---

### Set input directories

Set train, validation and test paths

In [8]:
my_data_dir = 'inputs/cherry_dataset/cherry-leaves'
train_path = my_data_dir + '/train'
val_path = my_data_dir + '/validation'
test_path = my_data_dir + '/test'

### Set output directory

In [6]:
version = 'v1'
file_path = f'outputs/{version}'

if 'outputs' in os.listdir(work_dir) and version in os.listdir(work_dir + '/outputs'):
    print('Old version already exists, please create a new version.')
    pass
else:
    os.makedirs(name=file_path)

Old version already exists, please create a new version.


### Set labels

In [9]:
labels = os.listdir(train_path)

print(
    f"Project Labels: {labels}"
)

Project Labels: ['healthy', 'powdery_mildew']


### Set image shape

Import the image shape from the output folder using the library joblib


In [10]:
import joblib
version = 'v1'
image_shape = joblib.load(filename=f"outputs/{version}/image_shape.pkl")
image_shape

(50, 50, 3)